In [ ]:
# bibliotecas
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader, ConcatDataset
import numpy as np
import kagglehub
from collections import Counter
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

### Leitura do dataset

In [2]:
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

In [ ]:
train_path = f"{path}/cifake/train"
test_path = f"{path}/cifake/test"

### Pré-processamento

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root="../data/train", transform=transform)
test_data = datasets.ImageFolder(root="../data/test", transform=transform)

### Divisão Holdout

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_data.targets)), train_data.targets))

In [16]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [17]:
print(f"Treino: {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste: {len(test_data)} imagens")

Treino: 70000 imagens
Validação: 30000 imagens
Teste: 20000 imagens


In [ ]:
train_labels = [label for _, label in train_data]

In [ ]:
def get_X_y(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.numpy())  
        y.append(label)
    X = np.stack(X)
    y = np.array(y)
    return X, y

In [ ]:
X_train, y_train = get_X_y(train_split)
X_val, y_val = get_X_y(val_split)
X_test, y_test = get_X_y(test_data)

## Treino

In [ ]:
device = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print("Usando:", device)

def preprocess_keras(X):
    # transpor de (N, C, H, W) -> (N, H, W, C)
    X = np.transpose(X, (0, 2, 3, 1))
    X = preprocess_input(X.astype(np.float32))  # normalização ResNet50
    return X

X_train_keras = preprocess_keras(X_train)
X_val_keras = preprocess_keras(X_val)
X_test_keras = preprocess_keras(X_test)


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # pra transfer Learning congela base


model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train_keras, y_train,
    validation_data=(X_val_keras, y_val),
    epochs=10,
    batch_size=32,
    verbose=3
)


In [ ]:
y_pred_prob = model.predict(X_test_keras).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC (ROC): {roc_auc:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title("Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()
plt.grid()
plt.show()